# Parsing LLM Output
This notebook takes output from finetuned Llama 65B and parses it to prepare a dataset that will be used for RLHF. 

## Load in output and parse it
This block loads in the output, parses it for solution and question, and executes the solution block. It inlcudes Try and Except blocks for cases when the model creates an incomplete output.

In [155]:
from pprint import pprint
file_path = 'testing_output.txt'
#data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("### Response: ")
    del sections[0]  # Remove the initial part before the first question

    for section in sections:
        try:
            question = section.split("Question:")[1].strip()
            question = section.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            solution = solution.split('\n    return result')[0]
            solution = solution + '\n    return result'
            solution_text = solution
            try:
                # Execute the Python code
                exec(solution)
                answer = solution()
                data.append({"question": question, "solution": solution_text, 'answer': answer})
            except: 
                try: #pick up questions that did not indent the code block
                    solution_new = ""
                    lines = solution.split('\n')
                    indent = False
                    for line in lines:
                        if line.startswith('def solution():'):
                            solution_new += line + '\n'
                            
                        if not line.startswith('def solution():') and not line.startswith('    return result'):
                            solution_new += '    ' + line + '\n'
                            
                        if line.startswith('    return result'):
                            solution_new += line
                    solution_text = solution_new
                    # Execute the Python code
                    exec(solution_new)
                    answer = solution()
                    data.append({"question": question, "solution": solution_text, 'answer': answer})
                
                except: 
                    pass
                    #pass if code doesn't execute
        except IndexError:
            pass
            #pass if there is no solution

In [156]:
len(data)

586

## Export as JSON

In [157]:
import json
data_path = 'app/questions.json'

# Save data as a JSON file
with open(data_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print("Data saved as JSON successfully.")

Data saved as JSON successfully.


# Export Data as Text Files for Zooniverse

In [87]:
questions_list = []
for i in range(0, len(data)):
    questions_list.append("Question: " + data[i]['question'] + "\n" + "Solution: " + data[i]['solution'] + "Answer: " + str(data[i]['answer']))
for i in range(0, len(questions_list)):
    #write code to create new text file for each question

['Question: The Culinary Institute of America, the best culinary school in the world, opened a new class. 40 people were enrolled in the class. The class ran for 6 months. Each student paid $3000 per month. How much money did the school make from this class?\nSolution: def solution():\n    #40 people paid $3000 for 6 months.\n    class_size = 40\n    #6 months of schooling cost $3000 a month.\n    tuition = 3000\n    class_length = 6\n    result = class_size * tuition * class_length\n    \n    return result\nAnswer: 720000',
 "Question: Melanie has $45. She shops at a grocery store for 2 hours and spends $18. She then goes to a bakery and spends $12. Melanie is thirsty and so she buys a soda for $3. She now has no money left. How much money did Melanie spend per hour at the grocery store?\nSolution: def solution():\n    #Melanie started with $45\n    money_initial = 45\n    #Melanie spent $18 at the grocery store\n    grocery_store_spent = 18\n    #Melanie spent $12 at the bakery\n    

In [84]:
len(data)

8